In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, minmax_scale, MinMaxScaler, StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout, Activation, Flatten
from sklearn.metrics import ConfusionMatrixDisplay
import warnings
warnings.filterwarnings("ignore")

In [2]:
astro = pd.read_csv("E:/Python Programs/Astro_EuroMillions_1.csv")
astro.head()

,Data,Day,Nakshatra,Pada-1,Pada-2,Pada-3,Pada-4,1st Num,2nd Num,3rd Num,4th Num,5th Num,1st Lucky Star,2nd Lucky Star,Winning Numbers
0,Train,Tuesday,Ashwini,NaN,NaN,NaN,3.0,7,15,18,46,49,10,12,7 15 18 46 49 10 12
1,Train,Tuesday,Bharani,NaN,NaN,NaN,NaN,7,15,18,46,49,10,12,7 15 18 46 49 10 12
2,Train,Tuesday,Krittika,NaN,NaN,NaN,NaN,7,15,18,46,49,10,12,7 15 18 46 49 10 12
3,Train,Tuesday,Rohini,NaN,NaN,NaN,NaN,7,15,18,46,49,10,12,7 15 18 46 49 10 12
4,Train,Tuesday,Mrigashirsha,NaN,NaN,NaN,NaN,7,15,18,46,49,10,12,7 15 18 46 49 10 12


In [3]:
astro.fillna(value={"Pada-1":0, "Pada-2":0, "Pada-3":0, "Pada-4":0},inplace=True)

In [4]:
astro["Pada-1"] = astro["Pada-1"].astype(dtype='int32')
astro["Pada-2"] = astro["Pada-2"].astype(dtype='int32')
astro["Pada-3"] = astro["Pada-3"].astype(dtype='int32')
astro["Pada-4"] = astro["Pada-4"].astype(dtype='int32')

In [5]:
encoding = LabelEncoder()
astro["Day"] = encoding.fit_transform(astro["Day"])
astro["Nakshatra"] = encoding.fit_transform(astro["Nakshatra"])

In [6]:
astro.head()

,Data,Day,Nakshatra,Pada-1,Pada-2,Pada-3,Pada-4,1st Num,2nd Num,3rd Num,4th Num,5th Num,1st Lucky Star,2nd Lucky Star,Winning Numbers
0,Train,1,3,0,0,0,3,7,15,18,46,49,10,12,7 15 18 46 49 10 12
1,Train,1,4,0,0,0,0,7,15,18,46,49,10,12,7 15 18 46 49 10 12
2,Train,1,9,0,0,0,0,7,15,18,46,49,10,12,7 15 18 46 49 10 12
3,Train,1,19,0,0,0,0,7,15,18,46,49,10,12,7 15 18 46 49 10 12
4,Train,1,11,0,0,0,0,7,15,18,46,49,10,12,7 15 18 46 49 10 12


In [7]:
astro_train = astro[astro['Data'] == 'Train']
astro_test = astro[astro['Data'] == 'Test']

In [8]:
astro_train.head()

,Data,Day,Nakshatra,Pada-1,Pada-2,Pada-3,Pada-4,1st Num,2nd Num,3rd Num,4th Num,5th Num,1st Lucky Star,2nd Lucky Star,Winning Numbers
0,Train,1,3,0,0,0,3,7,15,18,46,49,10,12,7 15 18 46 49 10 12
1,Train,1,4,0,0,0,0,7,15,18,46,49,10,12,7 15 18 46 49 10 12
2,Train,1,9,0,0,0,0,7,15,18,46,49,10,12,7 15 18 46 49 10 12
3,Train,1,19,0,0,0,0,7,15,18,46,49,10,12,7 15 18 46 49 10 12
4,Train,1,11,0,0,0,0,7,15,18,46,49,10,12,7 15 18 46 49 10 12


In [9]:
astro_train.drop(labels='Data', axis=1, inplace=True)

In [10]:
astro_train.head()

,Day,Nakshatra,Pada-1,Pada-2,Pada-3,Pada-4,1st Num,2nd Num,3rd Num,4th Num,5th Num,1st Lucky Star,2nd Lucky Star,Winning Numbers
0,1,3,0,0,0,3,7,15,18,46,49,10,12,7 15 18 46 49 10 12
1,1,4,0,0,0,0,7,15,18,46,49,10,12,7 15 18 46 49 10 12
2,1,9,0,0,0,0,7,15,18,46,49,10,12,7 15 18 46 49 10 12
3,1,19,0,0,0,0,7,15,18,46,49,10,12,7 15 18 46 49 10 12
4,1,11,0,0,0,0,7,15,18,46,49,10,12,7 15 18 46 49 10 12


In [11]:
astro_test.head()

,Data,Day,Nakshatra,Pada-1,Pada-2,Pada-3,Pada-4,1st Num,2nd Num,3rd Num,4th Num,5th Num,1st Lucky Star,2nd Lucky Star,Winning Numbers
966,Test,0,3,0,0,0,6,6,9,10,30,49,3,4,6 9 10 30 49 3 4
967,Test,0,4,0,1,0,0,6,9,10,30,49,3,4,6 9 10 30 49 3 4
968,Test,0,9,0,3,0,0,6,9,10,30,49,3,4,6 9 10 30 49 3 4
969,Test,0,19,0,0,0,0,6,9,10,30,49,3,4,6 9 10 30 49 3 4
970,Test,0,11,0,0,0,0,6,9,10,30,49,3,4,6 9 10 30 49 3 4


In [12]:
astro_test.drop(labels='Data', axis=1, inplace=True)

In [13]:
astro_test.head()

,Day,Nakshatra,Pada-1,Pada-2,Pada-3,Pada-4,1st Num,2nd Num,3rd Num,4th Num,5th Num,1st Lucky Star,2nd Lucky Star,Winning Numbers
966,0,3,0,0,0,6,6,9,10,30,49,3,4,6 9 10 30 49 3 4
967,0,4,0,1,0,0,6,9,10,30,49,3,4,6 9 10 30 49 3 4
968,0,9,0,3,0,0,6,9,10,30,49,3,4,6 9 10 30 49 3 4
969,0,19,0,0,0,0,6,9,10,30,49,3,4,6 9 10 30 49 3 4
970,0,11,0,0,0,0,6,9,10,30,49,3,4,6 9 10 30 49 3 4


In [14]:
astro_test.reset_index(inplace=True, drop=True)

In [15]:
astro_test.head()

,Day,Nakshatra,Pada-1,Pada-2,Pada-3,Pada-4,1st Num,2nd Num,3rd Num,4th Num,5th Num,1st Lucky Star,2nd Lucky Star,Winning Numbers
0,0,3,0,0,0,6,6,9,10,30,49,3,4,6 9 10 30 49 3 4
1,0,4,0,1,0,0,6,9,10,30,49,3,4,6 9 10 30 49 3 4
2,0,9,0,3,0,0,6,9,10,30,49,3,4,6 9 10 30 49 3 4
3,0,19,0,0,0,0,6,9,10,30,49,3,4,6 9 10 30 49 3 4
4,0,11,0,0,0,0,6,9,10,30,49,3,4,6 9 10 30 49 3 4


In [16]:
astro_train.shape, astro_test.shape

((993, 14), (527, 14))

In [18]:
X = astro_train[["Day","Nakshatra","Pada-1","Pada-2","Pada-3","Pada-4"]]
X.head()

,Day,Nakshatra,Pada-1,Pada-2,Pada-3,Pada-4
0,1,3,0,0,0,3
1,1,4,0,0,0,0
2,1,9,0,0,0,0
3,1,19,0,0,0,0
4,1,11,0,0,0,0


In [19]:
scaler = StandardScaler().fit(X.values)
X_transformed = scaler.transform(X.values)
X_train = pd.DataFrame(data=X_transformed, index=X.index, columns=["Day","Nakshatra","Pada-1","Pada-2","Pada-3","Pada-4"])

In [20]:
X_train.head()

,Day,Nakshatra,Pada-1,Pada-2,Pada-3,Pada-4
0,0.947978,-1.299411,-0.255879,-0.272398,-0.238277,1.492186
1,0.947978,-1.170431,-0.255879,-0.272398,-0.238277,-0.283063
2,0.947978,-0.525531,-0.255879,-0.272398,-0.238277,-0.283063
3,0.947978,0.764268,-0.255879,-0.272398,-0.238277,-0.283063
4,0.947978,-0.267572,-0.255879,-0.272398,-0.238277,-0.283063


In [21]:
y1 = astro_train[["1st Num"]]
y1.head()

,1st Num
0,7
1,7
2,7
3,7
4,7


In [22]:
scaler = StandardScaler().fit(y1.values)
y1_transformed = scaler.transform(y1.values)
y1_train = pd.DataFrame(data=y1_transformed, index=y1.index, columns=["1st Num"])

In [23]:
y1_train.head()

,1st Num
0,-0.394485
1,-0.394485
2,-0.394485
3,-0.394485
4,-0.394485


In [24]:
X_t = astro_test[["Day","Nakshatra","Pada-1","Pada-2","Pada-3","Pada-4"]]
X_t.head()

,Day,Nakshatra,Pada-1,Pada-2,Pada-3,Pada-4
0,0,3,0,0,0,6
1,0,4,0,1,0,0
2,0,9,0,3,0,0
3,0,19,0,0,0,0
4,0,11,0,0,0,0


In [25]:
scaler = StandardScaler().fit(X_t.values)
X_t_transformed = scaler.transform(X_t.values)
X_test = pd.DataFrame(data=X_t_transformed, index=X_t.index, columns=["Day","Nakshatra","Pada-1","Pada-2","Pada-3","Pada-4"])

In [26]:
X_test.head()

,Day,Nakshatra,Pada-1,Pada-2,Pada-3,Pada-4
0,-0.950014,-1.289071,-0.264237,-0.303522,-0.257065,4.021985
1,-0.950014,-1.159649,-0.264237,0.244272,-0.257065,-0.227102
2,-0.950014,-0.512534,-0.264237,1.339860,-0.257065,-0.227102
3,-0.950014,0.781694,-0.264237,-0.303522,-0.257065,-0.227102
4,-0.950014,-0.253688,-0.264237,-0.303522,-0.257065,-0.227102


In [27]:
y1_t = astro_test[["1st Num"]]
y1_t.head()

,1st Num
0,6
1,6
2,6
3,6
4,6


In [28]:
scaler = StandardScaler().fit(y1_t.values)
y1_t_transformed = scaler.transform(y1_t.values)
y1_test = pd.DataFrame(data=y1_t_transformed, index=y1_t.index, columns=["1st Num"])

In [29]:
y1_test.head()

,1st Num
0,-0.48962
1,-0.48962
2,-0.48962
3,-0.48962
4,-0.48962


In [30]:
X_train.shape, y1_train.shape, X_test.shape, y1_test.shape

((993, 6), (993, 1), (527, 6), (527, 1))

In [31]:
X_train_nmpy = np.array(X_train.values)
X_train_nmpy=np.reshape(X_train_nmpy,(993,6,1))
print(X_train_nmpy.shape)
X_train_nmpy

(993, 6, 1)


array([[[ 0.94797762],
        [-1.29941089],
        [-0.2558789 ],
        [-0.27239801],
        [-0.23827664],
        [ 1.49218644]],

       [[ 0.94797762],
        [-1.17043098],
        [-0.2558789 ],
        [-0.27239801],
        [-0.23827664],
        [-0.28306252]],

       [[ 0.94797762],
        [-0.52553144],
        [-0.2558789 ],
        [-0.27239801],
        [-0.23827664],
        [-0.28306252]],

       ...,

       [[ 0.94797762],
        [ 0.24834802],
        [-0.2558789 ],
        [ 5.04438277],
        [-0.23827664],
        [-0.28306252]],

       [[ 0.94797762],
        [ 1.40916721],
        [-0.2558789 ],
        [-0.27239801],
        [-0.23827664],
        [ 2.08393609]],

       [[ 0.94797762],
        [ 0.63528775],
        [-0.2558789 ],
        [-0.27239801],
        [-0.23827664],
        [-0.28306252]]])

In [32]:
y1_train_nmpy = np.array(y1_train.values)
y1_train_nmpy=np.reshape(y1_train_nmpy,(993,1))
print(y1_train_nmpy.shape)
y1_train_nmpy

(993, 1)


array([[-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.39448516],
       [-0.80295624],
       [-0.80295624],
       [-0.80295624],
       [-0.80295624],
       [-0.80295624],
       [-0.80295624],
       [-0.80295624],
       [-0.80295624],
       [-0.80295624],
       [-0.80295624],
       [-0.80295624],
       [-0.80295624],
       [-0.80295624],
       [-0.80295624],
       [-0.80295624],
       [-0.80295624],
       [-0.80295624],
       [-0.80295624],
       [-0

In [50]:
model = Sequential()

model.add(Flatten(input_shape=(6,1)))

model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Dense(400))
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Dense(800))
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Dense(1200))
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Dense(1600))
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Dense(2400))
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Dense(7))
model.add(Activation('softmax'))


model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_7 (Flatten)                  │ (None, 6)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_42 (Dense)                     │ (None, 200)                 │           1,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_42 (Activation)           │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_36 (Dropout)                 │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_43 (Dense)                     │ (None, 400)                 │          80,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_43 (Activation)           │ (None, 400)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_37 (Dropout)                 │ (None, 400)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_44 (Dense)                     │ (None, 800)                 │         320,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_44 (Activation)           │ (None, 800)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_38 (Dropout)                 │ (None, 800)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_45 (Dense)                     │ (None, 1200)                │         961,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_45 (Activation)           │ (None, 1200)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_39 (Dropout)                 │ (None, 1200)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_46 (Dense)                     │ (None, 1600)                │       1,921,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_46 (Activation)           │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_40 (Dropout)                 │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_47 (Dense)                     │ (None, 2400)                │       3,842,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_47 (Activation)           │ (None, 2400)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_41 (Dropout)                 │ (None, 2400)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_48 (Dense)                     │ (None, 7)                   │          16,807 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_48 (Activation)           │ (None, 7)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 7,144,607 (27.25 MB)

 Trainable params: 7,144,607 (27.25 MB)

 Non-trainable params: 0 (0.00 B)

In [51]:
model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])

In [52]:
model.fit(X_train_nmpy, y1_train_nmpy, batch_size=64, epochs=10, validation_split=0.2)

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - accuracy: 0.0000e+00 - loss: 1.0426 - val_accuracy: 0.0000e+00 - val_loss: 0.9006
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.0000e+00 - loss: 1.0742 - val_accuracy: 0.0000e+00 - val_loss: 0.9006
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - accuracy: 0.0000e+00 - loss: 0.9806 - val_accuracy: 0.0000e+00 - val_loss: 0.9006
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.0000e+00 - loss: 1.0165 - val_accuracy: 0.0000e+00 - val_loss: 0.9006
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.0000e+00 - loss: 1.0613 - val_accuracy: 0.0000e+00 - val_loss: 0.9006
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.0000e+00 - loss: 1.0769 - val_accuracy: 0.0000e+00 - val_loss: 0.9006
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - accuracy: 0.0000e+00 - loss: 1.0245 - val_accuracy: 0.0000e+00 - val_loss: 0.9006
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - ac